In [ ]:
from pathlib import Path
import subprocess


def run_dcm2nii_over_bids(
    bids_root: str | Path,
    dcm2nii_cmd: str = "dcm2niix",
) -> None:
    """
    Convert every /DCM folder in a BIDS tree to NIfTI and drop
    the output in the matching /ANAT folder.

    Layout expected
    ---------------
    <bids_root>/
        sub-XXX/
            ses-YYY/
                DCM/      ← raw DICOM files (already organised)
                ANAT/     ← will receive the NIfTI output

    Parameters
    ----------
    bids_root : str | Path
        Path to the BIDS directory.
    dcm2nii_cmd : str
        Executable for DICOM→NIfTI conversion (dcm2niix/dcm2nii).
    """
    bids_root = Path(bids_root).expanduser().resolve()

    # locate every DCM directory: sub-*/ses-*/DCM
    for dcm_dir in bids_root.glob("sub-*/ses-*/DCM"):
        if not dcm_dir.is_dir():
            continue

        anat_dir = dcm_dir.parent / "ANAT"
        anat_dir.mkdir(exist_ok=True)

        # run the converter; raise if it fails
        try:
            subprocess.run(
                [dcm2nii_cmd, "-b", "y", "-z", "y", "-o", anat_dir, str(dcm_dir)],
                check=True,
            )
        except subprocess.CalledProcessError as e:
            print(f"Error processing {dcm_dir}: {e}")
            continue

        # OPTIONAL — rename first *.nii* to sub-XXX_T1.nii.gz
        nifti_files = sorted(anat_dir.glob("*.nii*"))
        if len(nifti_files) == 1:
            subj_id = dcm_dir.parents[1].name.lstrip("sub-")
            nifti_files[0].rename(anat_dir / f"{subj_id}_T1.nii.gz")


In [ ]:
run_dcm2nii_over_bids('/Users/cu135/Partners HealthCare Dropbox/Calvin Howard/resources/datasets/MSA_Atrophy_Kurana/BIDS', 'dcm2niix')